In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flight-delays-fall-2018/sample_submission.csv.zip
/kaggle/input/flight-delays-fall-2018/flight_delays_train.csv.zip
/kaggle/input/flight-delays-fall-2018/flight_delays_test.csv.zip


# Ideas
1. Encode region rather than airport(?)
2. Drop month day since I rarely forsee month day having an impact on delay (except for holiday)
3. Assuming Northern Hemisphere, could do season df
4. Try to use less than 40 features(?). Simpler model --> less likely to overtrain.
5. DON'T TRAIN AND VALIDATE ON THE SAME DATA

In [2]:
pip install airportsdata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.3/912.3 kB 10.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import airportsdata

In [4]:
sample_sub = pd.read_csv("/kaggle/input/flight-delays-fall-2018/sample_submission.csv.zip")
train_df = pd.read_csv("/kaggle/input/flight-delays-fall-2018/flight_delays_train.csv.zip")
test_df = pd.read_csv("/kaggle/input/flight-delays-fall-2018/flight_delays_test.csv.zip")

EDA

In [5]:
display(train_df.head(), test_df.head())

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


Look at ways to filter columns and reduce dimensions of OHE

In [6]:
display(train_df.drop("dep_delayed_15min", axis = 1).head(), test_df.head())

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732
1,c-4,c-20,c-3,1548,US,PIT,MCO,834
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423


,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


In [7]:
import pandas as pd
import airportsdata

# Example preprocessing function (adjust as needed)
def preprocessing(input_df):
    df = input_df
    
    def add_int_region_elev(df):
        airports = airportsdata.load('IATA')

        O_international = []
        O_subd = []
        O_elevation = []

        D_international = []
        D_subd = []
        D_elevation = []

        same_region = []

        for _, row in df.iterrows():
            Origin = row["Origin"]
            Dest = row["Dest"]

            O_metadata = airports.get(Origin, {})
            D_metadata = airports.get(Dest, {})

            O_subd_value = O_metadata.get("subd", None)
            D_subd_value = D_metadata.get("subd", None)

            O_subd.append(O_subd_value)
            O_international.append("int" in O_metadata.get("name", "").lower())
            O_elevation.append(O_metadata.get("elevation", None))

            D_subd.append(D_subd_value)
            D_international.append("int" in D_metadata.get("name", "").lower())
            D_elevation.append(D_metadata.get("elevation", None))

            same_region.append(O_subd_value == D_subd_value)

        df["O-Region"] = O_subd
        
        print(df["O-Region"].unique())
        
        df["O-International"] = O_international 
        df["O-Elevation"] = O_elevation

        df["D-Region"] = D_subd
        df["D-International"] = D_international 
        df["D-Elevation"] = D_elevation

        df["Same-Region"] = same_region

        new_df = df.drop(["Origin", "Dest"], axis = 1)

        return new_df

    def ohe_func(df):    
        # One-Hot-Encode Categorical Variables
        month_ohe = pd.get_dummies(df.Month, prefix='Month').astype(int)
        month_ohe.drop(month_ohe.columns[0], axis=1, inplace=True)

        day_week_ohe = pd.get_dummies(df.DayOfWeek, prefix='DayOfWeek').astype(int)
        day_week_ohe.drop(day_week_ohe.columns[0], axis=1, inplace=True)

#         carrier_ohe = pd.get_dummies(df.UniqueCarrier, prefix='UniqueCarrier').astype(int)
#         carrier_ohe.drop(carrier_ohe.columns[0], axis=1, inplace=True)

        o_region_ohe = pd.get_dummies(df["O-Region"], prefix='O-Region').astype(int)
        o_region_ohe.drop(o_region_ohe.columns[0], axis=1, inplace=True)

        d_region_ohe = pd.get_dummies(df["D-Region"], prefix='D-Region').astype(int)
        d_region_ohe.drop(d_region_ohe.columns[0], axis=1, inplace=True)

        o_international_ohe = pd.get_dummies(df["O-International"], prefix='O-International').astype(int)
        d_international_ohe = pd.get_dummies(df["D-International"], prefix='D-International').astype(int)
        same_region_ohe = pd.get_dummies(df["Same-Region"], prefix='Same-Region').astype(int)

        return (month_ohe, day_week_ohe, o_region_ohe, d_region_ohe,
                o_international_ohe, d_international_ohe, same_region_ohe)
    
    def handle_low_frequency_columns(df, prefix, percentage):
        column_sums = df.sum()
        columns_percentage = (column_sums.sort_values(ascending=False) / df.shape[0]) * 100
        columns_to_aggregate = columns_percentage[columns_percentage < percentage].index

        other_column_name = f'{prefix}-Other'
        df[other_column_name] = df[columns_to_aggregate].sum(axis=1)
        df.drop(columns=columns_to_aggregate, inplace=True)

        return df
    
    df = add_int_region_elev(df)
    (month_ohe, day_week_ohe, o_region_ohe, d_region_ohe,
     o_international_ohe, d_international_ohe, same_region_ohe) = ohe_func(df)
    
    o_region_ohe = handle_low_frequency_columns(o_region_ohe, "O-Region", 8)
    d_region_ohe = handle_low_frequency_columns(d_region_ohe, "D-Region", 8)
    
    df_numeric = df[["DepTime", "Distance", "O-Elevation", "D-Elevation"]]

    combined_df = pd.concat([df_numeric, month_ohe, day_week_ohe,
                             o_region_ohe, d_region_ohe, o_international_ohe,
                             d_international_ohe, same_region_ohe], axis=1)
    
    # We'll see about fillna(). Check which columns contain na values
    combined_df = combined_df.fillna(0)

    return combined_df

In [8]:
# Combine train and test data
combined = pd.concat([train_df.drop("dep_delayed_15min", axis=1), test_df], axis=0)
combined.reset_index(drop=True, inplace=True)

# Apply preprocessing to the combined DataFrame
lean_mod_combined = preprocessing(combined)

# Separate the processed train and test data
train_len = train_df.shape[0]
lean_mod_train = lean_mod_combined.iloc[:train_len, :].reset_index(drop=True)
lean_mod_test = lean_mod_combined.iloc[train_len:, :].reset_index(drop=True)

# Display results
print("Processed Train Data:")
display(lean_mod_train.head())
print("Processed Test Data:")
display(lean_mod_test.head())

['Georgia' 'Pennsylvania' 'North Carolina' 'Colorado' 'Illinois'
 'Tennessee' 'Florida' 'Minnesota' 'California' 'Connecticut' 'Arizona'
 'Nevada' 'Texas' 'Iowa' 'Ohio' 'Virginia' 'Utah' 'Dist. Of Columbia'
 'Hawaii' 'New York' 'South Carolina' 'Washington' 'Arkansas' 'Oklahoma'
 'Michigan' 'Idaho' 'Missouri' 'Maryland' 'Alabama' 'Massachusetts'
 'Wyoming' 'Indiana' 'New Jersey' 'Alaska' 'Oregon' 'Wisconsin'
 'North Dakota' 'Louisiana' 'Kentucky' 'Maine' 'Rhode Island'
 'Mississippi' 'Montana' 'Kansas' 'Puerto Rico' 'New Hampshire' 'Nebraska'
 'New Mexico' 'Vermont' 'South Dakota' 'West Virginia' None
 'Virgin Islands' 'Delaware']
Processed Train Data:


,DepTime,Distance,O-Elevation,D-Elevation,Month_c-10,Month_c-11,Month_c-12,Month_c-2,Month_c-3,Month_c-4,...,O-Region-Other,D-Region_California,D-Region_Texas,D-Region-Other,O-International_False,O-International_True,D-International_False,D-International_True,Same-Region_False,Same-Region_True
0,1934,732,1026.2,606.4,0,0,0,0,0,0,...,1,0,1,0,0,1,0,1,1,0
1,1548,834,1202.9,96.4,0,0,0,0,0,1,...,1,0,0,1,0,1,0,1,1,0
2,1422,416,435.2,799.5,0,0,0,0,0,0,...,1,0,0,1,0,1,0,1,1,0
3,1015,872,5433.8,340.9,0,1,0,0,0,0,...,1,0,0,1,0,1,0,1,1,0
4,1828,423,619.8,984.5,1,0,0,0,0,0,...,1,0,0,1,0,1,1,0,1,0


Processed Test Data:


,DepTime,Distance,O-Elevation,D-Elevation,Month_c-10,Month_c-11,Month_c-12,Month_c-2,Month_c-3,Month_c-4,...,O-Region-Other,D-Region_California,D-Region_Texas,D-Region-Other,O-International_False,O-International_True,D-International_False,D-International_True,Same-Region_False,Same-Region_True
0,615,598,256.6,1134.8,0,0,0,0,0,0,...,0,0,0,1,1,0,0,1,1,0
1,739,1235,2181.2,46.3,0,0,0,0,0,1,...,1,0,1,0,0,1,1,0,1,0
2,651,577,963.6,680.0,0,0,1,0,0,0,...,1,0,0,1,0,1,0,1,1,0
3,1614,377,143.4,266.4,0,0,0,0,1,0,...,1,0,0,1,0,1,1,0,1,0
4,1505,258,680.0,617.3,0,0,0,0,0,0,...,1,0,0,1,0,1,0,1,1,0


# Modeling

Classification Problem: SGD, Decision Forest, Boosted Decision Tree, Neural Network

Random Forest

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score, KFold

In [10]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

X_train = lean_mod_train
y_train = train_df["dep_delayed_15min"].map({"Y":1, "N":0})


# forest = RandomForestClassifier(max_depth = 8, max_features = 32, n_jobs=-1, random_state=17)

# forest_params = {"n_estimators": [115]
# }

# forest_grid = GridSearchCV(forest, forest_params, cv=5, n_jobs=-1, verbose=True, scoring='roc_auc')
# forest_grid.fit(X_train, y_train)

# forest_grid.best_params_, forest_grid.best_score_

In [11]:
forest_optimal = RandomForestClassifier(n_estimators = 115, max_depth = 8, max_features = 33, random_state=17)
forest_optimal.fit(X_train, y_train)

RandomForestClassifier(max_depth=8, max_features=33, n_estimators=115,
                       random_state=17)

In [12]:
test_predictions = forest_optimal.predict_proba(lean_mod_test)[:, 1]

In [13]:
predictions_df = pd.DataFrame({
    'id': np.arange(0, len(test_predictions)),
    'dep_delayed_15min': test_predictions})
predictions_df

,id,dep_delayed_15min
0,0,0.024542
1,1,0.063543
2,2,0.036993
3,3,0.208250
4,4,0.234991
...,...,...
99995,99995,0.068186
99996,99996,0.201766
99997,99997,0.209418
99998,99998,0.080790


Submission

In [14]:
predictions_df.to_csv('submission.csv', index=False)